## **PROJECT - Classifying Colored Images of Cats and Dogs**

#### Details
- Convert the dimensions of the images to 28 x 28 pixels.
- The dataset consists of:
  - 2000 training images (1000 cats and 1000 dogs)
  - 1000 validation images (500 cats and 500 dogs)
  - 1000 test images (500 cats and 500 dogs)
- We will use data augmentation to improve model performance through various transformations.
- Further improvements in model performance will be achieved using transfer earning.


In [1]:
# Import the relevant libraries
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
# Read and load the data from their various directories
train_dir = r'C:\Users\HP\Downloads\cats_and_dogs_small\train'
validation_dir = r'C:\Users\HP\Downloads\cats_and_dogs_small\validation'
test_dir = r'C:\Users\HP\Downloads\cats_and_dogs_small\test'

### Data Preprocessing

#### Steps:
- Decode the jpeg images to rgb pixels grids
- Convert the grids into floating point tensors
- Rescale the pixel values to (0, 1) interval

In [4]:
# Import the library to perform all the preprocessing steps
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
# Generating batches of tensor image data, rescale, set target size and batch size

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [6]:
# Import VGGNet architecture for transfer learning

from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [7]:
conv_base.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 150, 150, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 150, 150, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 75, 75, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 75, 75, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 75, 75, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 37, 37, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 37, 37, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 18, 18, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 18, 18, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 9, 9, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Import the libraries for training the neural network
from tensorflow.keras import layers, models, optimizers

In [9]:
# Arranging the layers in sequence

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compiling
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(learning_rate=2e-5),
              metrics=['accuracy'])

# Use actual input data (dummy data) to build the model
dummy_data = np.random.random((20, 150, 150, 3))  # Use a batch of 20 samples with input shape (150, 150, 3)
model.predict(dummy_data)  # Perform a forward pass to build the model

# Now, print the layer names and output shapes using the `.output` property
for layer in model.layers:
    print(f"Layer: {layer.name}, Output Tensor: {layer.output}")  # This should work without error

# Print the model summary
model.summary()

1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
Layer: vgg16, Output Tensor: <KerasTensor shape=(None, 4, 4, 512), dtype=float32, sparse=False, name=keras_tensor_18>
Layer: flatten, Output Tensor: <KerasTensor shape=(20, 8192), dtype=float32, sparse=False, name=keras_tensor_38>
Layer: dense, Output Tensor: <KerasTensor shape=(20, 256), dtype=float32, sparse=False, name=keras_tensor_39>
Layer: dense_1, Output Tensor: <KerasTensor shape=(20, 1), dtype=float32, sparse=False, name=keras_tensor_40>


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (20, 4, 4, 512)        │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (20, 8192)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (20, 256)              │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (20, 1)                │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,812,353 (64.13 MB)

 Trainable params: 16,812,353 (64.13 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Reload the model weights for the first epoch

model.load_weights('CNN_Project_Model_resumed.weights.h5')

In [11]:
# Saving from the resumed training
checkpoint_cb = keras.callbacks.ModelCheckpoint('Cat_and_dogs_model_final.weights.h5', save_weights_only=True, save_best_only=False)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

In [12]:
val_data = next(validation_generator)
print(val_data[0].shape)  # Print shape of images
print(val_data[1].shape)  # Print shape of labels

(20, 150, 150, 3)
(20,)


In [13]:
# Fit the model

history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=3,
    validation_data=validation_generator,
    validation_steps=50,
    callbacks=[checkpoint_cb, early_stopping_cb],
    initial_epoch=2)

Epoch 3/3
100/100 ━━━━━━━━━━━━━━━━━━━━ 4743s 47s/step - accuracy: 0.9048 - loss: 0.2294 - val_accuracy: 0.9250 - val_loss: 0.1764


In [15]:
# Store the accuracies and losses in a dataframe

hist_df = pd.DataFrame(history.history)
hist_df.head()

,accuracy,loss,val_accuracy,val_loss
0,0.918,0.214497,0.925,0.176352


In [16]:
# Save the accuracies and losses to a *csv file

hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [18]:
# Generating batches of tensor image data from the test data, rescale, set target size and batch size

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 1000 images belonging to 2 classes.


In [19]:
# Check for the model accuracy from the test set

model.evaluate(test_generator, steps=50)

50/50 ━━━━━━━━━━━━━━━━━━━━ 507s 10s/step - accuracy: 0.9170 - loss: 0.1700


[0.1725403070449829, 0.9210000038146973]